In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score

# Data Preparation

In [2]:
data = pd.read_csv('train_test.csv')

In [3]:
data['los_icu_class'] = data['los_icu_class'].apply(lambda x: 0 if x == 'less than 3 days' else 1)

In [4]:
selected_cols = [col for col in data.columns if col not in ['charttime', 'hosp_admittime', 'hosp_dischtime', 'icu_intime', 'icu_outtime','los_icu','text_embeddings','los_icu_class']]

In [5]:
# fill the nan
df_full = data[selected_cols].groupby('id').transform(lambda x: x.fillna(x.mean()))
df = df_full.fillna(df_full.mean())

In [6]:
# split X,y
X = df
y = data['los_icu_class']

# numerical columns
num = df.select_dtypes(include=['float']).columns
X_num = X[num]

# min-max standardization
scaler = MinMaxScaler(feature_range=(0, 1))
X_num_scaled = scaler.fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, columns=num, index=X_num.index)
X_scaled = X.copy()
X_scaled[num] = X_num_scaled[num]

In [7]:
# set time step
time_steps = 25
X_scaled = np.array([X_scaled[i:i + time_steps] for i in range(0, len(X_scaled) - time_steps + 1, time_steps)])
y = np.array([y[i] for i in range(time_steps - 1, len(y), time_steps)])

# split train test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [8]:
# change the shape of data to fit the network requirement
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))
y_train = y_train.reshape(-1, 1)

# Modeling

In [9]:
# get the time stamp and num of features
time_steps = X_train.shape[1]
num_features = X_train.shape[2]

# tcn model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu',dilation_rate=1,input_shape=(time_steps, num_features)),
    #MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu',dilation_rate=2),
    #MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu',dilation_rate=4)
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1, activation='sigmoid')  # output layer, bi-classification
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.F1Score()])

model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

d:\Anaconda\envs\tensorflow\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - f1_score: 0.6398 - loss: 0.6941 - val_f1_score: 0.6445 - val_loss: 0.6842
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - f1_score: 0.6476 - loss: 0.6836 - val_f1_score: 0.6445 - val_loss: 0.6798
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - f1_score: 0.6345 - loss: 0.6768 - val_f1_score: 0.6445 - val_loss: 0.6784
Epoch 4/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - f1_score: 0.6449 - loss: 0.6768 - val_f1_score: 0.6445 - val_loss: 0.6770
Epoch 5/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - f1_score: 0.6361 - loss: 0.6700 - val_f1_score: 0.6445 - val_loss: 0.6773
Epoch 6/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - f1_score: 0.6449 - loss: 0.6702 - val_f1_score: 0.6445 - val_loss: 0.6832
Epoch 7/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - f1_score: 0.6443 - loss: 0.6706 - val_f1_score: 0.6445 - val_loss: 0.6741
Epoch 8/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - f1_score: 0.6451 - loss: 0.6582 - val_f1_score: 0.

In [10]:
# save the trained model
model.save('tcn_los.keras')

In [11]:
# on testing set
y_test_pred = model.predict(X_test)

# classify based on probability
y_pred_class = (y_test_pred > 0.5).astype(int)

# evaluate the result
precision = precision_score(y_test, y_pred_class)
recall = recall_score(y_test, y_pred_class)
f1 = f1_score(y_test, y_pred_class)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Precision: 0.530791788856305
Recall: 0.48460508701472554
F1 Score: 0.5066480055983205


In [12]:
X_scaled = np.concatenate((X_train, X_test), axis=0)
# predict the whole X and output probability
y_pred = model.predict(X_scaled)

result= pd.DataFrame(y_pred, columns=['probs'])
result.to_csv('tcn_probs_of_los_traintest.csv', index=False)

511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
